In [27]:
import pandas as pd
from jf.autocompute.jf import O
from collections import Counter

In [3]:
path = "output/results/setup_basic/refgrn_mtristate1_e90_s8_p_n1_t49.history.csv"
df = pd.read_csv(path)

In [4]:
df.head()

,Unnamed: 0,Tc_h,appear_time,child1,child2,division_time,index,type
0,0,33.204867,49.0,144.0,145.0,50.314751,0.0,RG
1,1,47.272241,49.0,102.0,103.0,49.949761,1.0,RG
2,2,39.500638,49.0,64.0,65.0,49.067002,2.0,RG
3,3,48.129028,49.0,104.0,105.0,49.796524,3.0,RG
4,4,50.698540,49.0,66.0,67.0,49.437449,4.0,RG


In [12]:
df.tail()

,Unnamed: 0,Tc_h,appear_time,child1,child2,division_time,index,type
18713,18713,72.603147,89.419458,NaN,NaN,92.444589,18713.0,IP
18714,18714,0.000000,89.263217,NaN,NaN,inf,18714.0,PostMitotic
18715,18715,0.000000,89.263217,NaN,NaN,inf,18715.0,PostMitotic
18716,18716,60855.675407,89.450273,NaN,NaN,2625.103415,18716.0,GP
18717,18717,69783.408792,89.450273,NaN,NaN,2997.092306,18717.0,GP


In [18]:
pd.isna(df.iloc[-1]["child1"])

True

In [39]:
class TmpCell:
    def __init__(self, appear_time, cell_type: str, children: list = []):
        if len(children) == 2:
            self._type = O(name="Progenitor")
        elif cell_type == "PostMitotic":
            self._type = O(name="PostMitotic")
        else:
            self._type = O(name="Cell")
        
        self.children = children
        self.appear_time = appear_time
        
    def type(self):
        return self._type

In [36]:
def build_population(df):
    population = dict()
    for i, row in df.iterrows():
        children = list(map(int, filter(lambda x: not pd.isna(x), [row["child1"], row["child2"]])))
        population[int(row["index"])] = TmpCell(row["appear_time"], row["type"], children)
        
    return population

In [5]:
def fate_corr(cell_pairs, kC="Progenitor", kN="PostMitotic"):
    counts = Counter(cell_pairs)
    
    CC, CN, NN = counts[(kC, kC)], counts[(kN, kC)] + counts[(kC, kN)], counts[(kN, kN)]
    T = CC + CN + NN
    pCC, pCN, pNN = CC / T, CN / T, NN / T
    all_C, all_N = 2 * CC + CN, 2 * NN + CN
    pC, pN = all_C / (T * 2), all_N / (T * 2)
    eCC, eCN, eNN = pC**2, 2*pC*pN, pN**2
    f_metric = 1 - pCN / eCN
    return f_metric

In [24]:
def get_fmetric_pairs(population, min_time=50, max_time=60):
    """ This function is only valid for GRNCells"""
    cells_in_window = [c for c in population.values() 
                       if c.appear_time >= min_time and c.appear_time < max_time]
    cells_divided = [c for c in cells_in_window if c.type().name == "Progenitor"]
    final_cells = list(filter(lambda x: all_daughter_defined(x, population), cells_divided))
    
    return pairs(final_cells, population)

In [7]:
def pairs(cells, population):
    return [tuple(population[x].type().name for x in cell.children) for cell in cells]

In [43]:
def all_daughter_defined(cell, population):
    """ Warning : the word Cell must correspond to an unknown type"""
    if not len(cell.children) == 2:
        return False
    
    for daughter_id in cell.children:
        if population[daughter_id].type().name == "Cell":
            return False
    return True

In [40]:
population = build_population(df)

In [41]:
cell_pairs = get_fmetric_pairs(population)

In [42]:
fate_corr(cell_pairs)

0.6756766016234643

In [45]:
a = dict(x=9, y=12)
b = dict(x=2, y=1)

In [49]:
res = dict(pd.DataFrame([a, b]).mean())

In [51]:
pd.DataFrame([res])

,x,y
0,5.5,6.5


In [56]:
pd.Series(res).to_csv("tmp.csv")

In [59]:
ad = pd.read_csv("tmp.csv", index_col=0)

In [60]:
ad

,0
x,5.5
y,6.5


In [64]:
dict(ad["0"])

{'x': 5.5, 'y': 6.5}